In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
cd ..

/home/suriya/dcu/Group-Project


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [6]:
from src.features.basic import features
from src.features.common import postproc_categorical, read_non_sensor_data
from src.featutils import combine
from src.dataset.tc4tl import make_datasets
import category_encoders as ce

trainset, devset, testset = make_datasets(combine(features, read_non_sensor_data),
                                          postproc_categorical, verbose=True, testing=0,
                                          tunables={ 
                                              "CategoricalEncoder" : ce.polynomial.PolynomialEncoder
                                          })

100%|████████████████████████████████████| 15552/15552 [00:42<00:00, 362.63it/s]


NO Pipe input given!


100%|████████████████████████████████████████| 935/935 [00:02<00:00, 370.60it/s]


Pipe input given!


100%|██████████████████████████████████████| 8423/8423 [00:21<00:00, 385.06it/s]


Pipe input given!


In [7]:
predictors = []
for col in trainset.columns:
    if "Rssi" in col or "Coarse" in col or "_" in col:
        predictors.append(col)
predictors

['RssiMin',
 'RssiPercentile:1',
 'RssiPercentile:5',
 'RssiPercentile:10',
 'RssiPercentile:25',
 'RssiPercentile:40',
 'RssiMean',
 'RssiPercentile:65',
 'RssiPercentile:80',
 'RssiPercentile:90',
 'RssiPercentile:95',
 'RssiPercentile:99',
 'RssiMax',
 'CoarseGrain',
 'Cat:TXDevice_0',
 'Cat:TXDevice_1',
 'Cat:TXDevice_2',
 'Cat:TXDevice_3',
 'Cat:TXDevice_4',
 'Cat:TXDevice_5',
 'Cat:TXDevice_6',
 'Cat:TXDevice_7',
 'Cat:TXDevice_8',
 'Cat:TXDevice_9',
 'Cat:TXDevice_10',
 'Cat:TXDevice_11',
 'Cat:TXDevice_12',
 'Cat:TXDevice_13',
 'Cat:TXPower_0',
 'Cat:TXPower_1',
 'Cat:RXDevice_0',
 'Cat:RXDevice_1',
 'Cat:RXDevice_2',
 'Cat:RXDevice_3',
 'Cat:RXDevice_4',
 'Cat:RXDevice_5',
 'Cat:RXDevice_6',
 'Cat:RXDevice_7',
 'Cat:RXDevice_8',
 'Cat:TXCarry_0',
 'Cat:RXCarry_0',
 'Cat:RXPose_0',
 'Cat:TXPose_0']

In [8]:
from src.models.evaluate import dual_evaluation
import pandas as pd
from src.ndcf import compute_ndcf

report, sysout = dual_evaluation(pd.concat([trainset, devset]), testset,
                    predictors=predictors,
                    target="Distance")
ndcf = compute_ndcf(sysout)
ndcf["nDCF"].mean()

0.22089693482325018

# Feature Selection

In [14]:
from sklearn.feature_selection import SelectFromModel

m0 = report["model:cg=0"]
trs0 = report["trainset:cg=0"]
model = SelectFromModel(m0, prefit=True)
X_new = model.transform(trs0[predictors])

/home/suriya/dcu/Group-Project/env/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [12]:
trs0.columns

Index(['RssiMin', 'RssiPercentile:1', 'RssiPercentile:5', 'RssiPercentile:10',
       'RssiPercentile:25', 'RssiPercentile:40', 'RssiMean',
       'RssiPercentile:65', 'RssiPercentile:80', 'RssiPercentile:90',
       'RssiPercentile:95', 'RssiPercentile:99', 'RssiMax', 'CoarseGrain',
       'fileid', 'Distance', 'DistanceFloat', 'Cat:TXDevice', 'Cat:TXPower',
       'Cat:RXDevice', 'Cat:TXCarry', 'Cat:RXCarry', 'Cat:RXPose',
       'Cat:TXPose', 'intercept', 'Cat:TXDevice_0', 'Cat:TXDevice_1',
       'Cat:TXDevice_2', 'Cat:TXDevice_3', 'Cat:TXDevice_4', 'Cat:TXDevice_5',
       'Cat:TXDevice_6', 'Cat:TXDevice_7', 'Cat:TXDevice_8', 'Cat:TXDevice_9',
       'Cat:TXDevice_10', 'Cat:TXDevice_11', 'Cat:TXDevice_12',
       'Cat:TXDevice_13', 'Cat:TXPower_0', 'Cat:TXPower_1', 'Cat:RXDevice_0',
       'Cat:RXDevice_1', 'Cat:RXDevice_2', 'Cat:RXDevice_3', 'Cat:RXDevice_4',
       'Cat:RXDevice_5', 'Cat:RXDevice_6', 'Cat:RXDevice_7', 'Cat:RXDevice_8',
       'Cat:TXCarry_0', 'Cat:RXCarry_0', 'C

## FS Example

In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris

iris = load_iris()
X = pd.DataFrame(iris.data, columns=["a", "b", "c", "d"])
#X = iris.data
y = iris.target
knn = KNeighborsClassifier(n_neighbors=4)

In [42]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

sfs1 = SFS(knn, 
           k_features=3, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=0)

sfs1 = sfs1.fit(X, y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished

[2021-11-09 19:32:25] Features: 1/3 -- score: 0.96[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished

[2021-11-09 19:32:25] Features: 2/3 -- score: 0.9733333333333334[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2021-11-09 19:32:25] Features: 3/3 -- score: 0.9733333333333334

In [43]:
sfs1.k_feature_names_

('b', 'c', 'd')

In [44]:
sfs1.subsets_

{1: {'feature_idx': (3,),
  'cv_scores': array([0.96]),
  'avg_score': 0.96,
  'feature_names': ('d',)},
 2: {'feature_idx': (2, 3),
  'cv_scores': array([0.97333333]),
  'avg_score': 0.9733333333333334,
  'feature_names': ('c', 'd')},
 3: {'feature_idx': (1, 2, 3),
  'cv_scores': array([0.97333333]),
  'avg_score': 0.9733333333333334,
  'feature_names': ('b', 'c', 'd')}}

## mlxtend.SFS

In [45]:
sfs = SFS(m0, 
           k_features=11, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=0)

sfs = sfs.fit(trs0[predictors], trs0["Distance"])


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  43 out of  43 | elapsed:    9.3s finished

[2021-11-09 19:34:39] Features: 1/11 -- score: 0.9994525547445255[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  42 out of  42 | elapsed:   20.3s finished

[2021-11-09 19:35:00] Features: 2/11 -- score: 1.0[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed:   16.1s finished

[2021-11-09 19:35:16] Features: 3/11 -- score: 1.0[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.

In [46]:
sfs.subsets_

{1: {'feature_idx': (6,),
  'cv_scores': array([0.99945255]),
  'avg_score': 0.9994525547445255,
  'feature_names': ('RssiMean',)},
 2: {'feature_idx': (6, 11),
  'cv_scores': array([1.]),
  'avg_score': 1.0,
  'feature_names': ('RssiMean', 'RssiPercentile:99')},
 3: {'feature_idx': (0, 6, 11),
  'cv_scores': array([1.]),
  'avg_score': 1.0,
  'feature_names': ('RssiMin', 'RssiMean', 'RssiPercentile:99')},
 4: {'feature_idx': (0, 1, 6, 11),
  'cv_scores': array([1.]),
  'avg_score': 1.0,
  'feature_names': ('RssiMin',
   'RssiPercentile:1',
   'RssiMean',
   'RssiPercentile:99')},
 5: {'feature_idx': (0, 1, 2, 6, 11),
  'cv_scores': array([1.]),
  'avg_score': 1.0,
  'feature_names': ('RssiMin',
   'RssiPercentile:1',
   'RssiPercentile:5',
   'RssiMean',
   'RssiPercentile:99')},
 6: {'feature_idx': (0, 1, 2, 3, 6, 11),
  'cv_scores': array([1.]),
  'avg_score': 1.0,
  'feature_names': ('RssiMin',
   'RssiPercentile:1',
   'RssiPercentile:5',
   'RssiPercentile:10',
   'RssiMean',
   

## Select from percentile features

In [49]:
from src.features.basic import percentile_features
from src.features.common import postproc_default
from src.dataset.tc4tl import make_datasets


trainset, devset, testset = make_datasets(percentile_features, postproc_default, verbose=True, testing=0, tunables={})

100%|█████████████████████████████████████| 8423/8423 [00:07<00:00, 1118.56it/s]


In [50]:
trainset

,P:0,P:1,P:2,P:3,P:4,P:5,P:6,P:7,P:8,P:9,...,P:96,P:97,P:98,P:min,P:max,P:mean,CoarseGrain,fileid,Distance,DistanceFloat
0,-79.00,-77.00,-76.09,-75.00,-75.0,-75.0,-75.0,-74.00,-74.0,-74.0,...,-51.00,-51.0,-50.0,-88.0,-49.0,-62.133482,0,aaadbuvp_tc4tl20.csv,1.8,1.8
1,-76.00,-67.42,-64.33,-64.00,-64.0,-64.0,-64.0,-63.52,-60.0,-60.0,...,-48.00,-48.0,-48.0,-76.0,-48.0,-55.255556,1,aaamkcii_tc4tl20.csv,3.0,3.0
2,-80.00,-78.00,-78.00,-77.00,-76.6,-76.0,-76.0,-75.00,-75.0,-75.0,...,-50.00,-49.0,-47.0,-89.0,-47.0,-62.289089,1,aabqtowt_tc4tl20.csv,1.2,1.2
3,-87.00,-86.00,-85.00,-84.00,-83.0,-83.0,-81.0,-80.00,-78.0,-78.0,...,-58.00,-57.0,-56.0,-93.0,-53.0,-67.938259,1,aadkjwss_tc4tl20.csv,1.8,1.8
4,-70.00,-67.00,-67.00,-66.00,-65.0,-65.0,-65.0,-64.00,-63.0,-63.0,...,-47.00,-46.0,-46.0,-72.0,-45.0,-55.760797,0,aafzrgzt_tc4tl20.csv,1.8,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15547,-84.00,-82.00,-79.00,-78.00,-77.0,-77.0,-77.0,-76.00,-75.0,-74.0,...,-51.00,-51.0,-50.0,-87.0,-50.0,-62.998435,1,zzwcoemu_tc4tl20.csv,1.2,1.2
15548,-92.00,-91.00,-90.07,-89.00,-89.0,-89.0,-87.0,-87.00,-87.0,-86.0,...,-66.00,-66.0,-65.0,-93.0,-64.0,-77.020677,1,zzwoqxuc_tc4tl20.csv,4.5,4.5
15549,-94.00,-91.00,-89.77,-88.36,-88.0,-87.0,-86.0,-85.00,-85.0,-85.0,...,-61.00,-61.0,-61.0,-99.0,-58.0,-74.469638,1,zzymwbbb_tc4tl20.csv,1.2,1.2
15550,-83.39,-82.00,-81.00,-80.00,-79.0,-78.0,-77.0,-76.00,-76.0,-75.0,...,-50.00,-50.0,-50.0,-88.0,-46.0,-64.321346,0,zzzflikz_tc4tl20.csv,1.8,1.8


In [51]:
predictors = []
for col in trainset.columns:
    if "P:" in col or "Coarse" in col:
        predictors.append(col)
predictors

['P:0',
 'P:1',
 'P:2',
 'P:3',
 'P:4',
 'P:5',
 'P:6',
 'P:7',
 'P:8',
 'P:9',
 'P:10',
 'P:11',
 'P:12',
 'P:13',
 'P:14',
 'P:15',
 'P:16',
 'P:17',
 'P:18',
 'P:19',
 'P:20',
 'P:21',
 'P:22',
 'P:23',
 'P:24',
 'P:25',
 'P:26',
 'P:27',
 'P:28',
 'P:29',
 'P:30',
 'P:31',
 'P:32',
 'P:33',
 'P:34',
 'P:35',
 'P:36',
 'P:37',
 'P:38',
 'P:39',
 'P:40',
 'P:41',
 'P:42',
 'P:43',
 'P:44',
 'P:45',
 'P:46',
 'P:47',
 'P:48',
 'P:49',
 'P:50',
 'P:51',
 'P:52',
 'P:53',
 'P:54',
 'P:55',
 'P:56',
 'P:57',
 'P:58',
 'P:59',
 'P:60',
 'P:61',
 'P:62',
 'P:63',
 'P:64',
 'P:65',
 'P:66',
 'P:67',
 'P:68',
 'P:69',
 'P:70',
 'P:71',
 'P:72',
 'P:73',
 'P:74',
 'P:75',
 'P:76',
 'P:77',
 'P:78',
 'P:79',
 'P:80',
 'P:81',
 'P:82',
 'P:83',
 'P:84',
 'P:85',
 'P:86',
 'P:87',
 'P:88',
 'P:89',
 'P:90',
 'P:91',
 'P:92',
 'P:93',
 'P:94',
 'P:95',
 'P:96',
 'P:97',
 'P:98',
 'P:min',
 'P:max',
 'P:mean',
 'CoarseGrain']

In [52]:
from src.models.evaluate import dual_evaluation
import pandas as pd
from src.ndcf import compute_ndcf

report, sysout = dual_evaluation(pd.concat([trainset, devset]), testset,
                    predictors=predictors,
                    target="Distance")
ndcf = compute_ndcf(sysout)
ndcf["nDCF"].mean()

0.49118898828857765

In [55]:
from sklearn.ensemble import RandomForestClassifier

In [56]:
trs0 = report["trainset:cg=0"]
m0 = report["model:cg=0"]

sfs = SFS(RandomForestClassifier(), 
           k_features=12, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=0)

sfs = sfs.fit(trs0[predictors], trs0["Distance"])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 103 out of 103 | elapsed:   24.9s finished

[2021-11-09 19:57:45] Features: 1/12 -- score: 0.9992700729927008[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 102 out of 102 | elapsed:   51.6s finished

[2021-11-09 19:58:36] Features: 2/12 -- score: 1.0[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 101 out of 101 | elapsed:   47.4s finished

[2021-11-09 19:59:24] Features: 3/12 -- score: 1.0[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.

In [57]:
sfs.subsets_

{1: {'feature_idx': (101,),
  'cv_scores': array([0.99927007]),
  'avg_score': 0.9992700729927008,
  'feature_names': ('P:mean',)},
 2: {'feature_idx': (0, 101),
  'cv_scores': array([1.]),
  'avg_score': 1.0,
  'feature_names': ('P:0', 'P:mean')},
 3: {'feature_idx': (0, 3, 101),
  'cv_scores': array([1.]),
  'avg_score': 1.0,
  'feature_names': ('P:0', 'P:3', 'P:mean')},
 4: {'feature_idx': (0, 1, 3, 101),
  'cv_scores': array([1.]),
  'avg_score': 1.0,
  'feature_names': ('P:0', 'P:1', 'P:3', 'P:mean')},
 5: {'feature_idx': (0, 1, 2, 3, 101),
  'cv_scores': array([1.]),
  'avg_score': 1.0,
  'feature_names': ('P:0', 'P:1', 'P:2', 'P:3', 'P:mean')},
 6: {'feature_idx': (0, 1, 2, 3, 4, 101),
  'cv_scores': array([1.]),
  'avg_score': 1.0,
  'feature_names': ('P:0', 'P:1', 'P:2', 'P:3', 'P:4', 'P:mean')},
 7: {'feature_idx': (0, 1, 2, 3, 4, 5, 101),
  'cv_scores': array([1.]),
  'avg_score': 1.0,
  'feature_names': ('P:0', 'P:1', 'P:2', 'P:3', 'P:4', 'P:5', 'P:mean')},
 8: {'feature_id